In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

23/12/27 13:57:40 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/27 13:57:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 13:57:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/27 13:57:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
iris = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/iris.csv", header=True, inferSchema=True, sep=",")
iris.show(5)
print(f"Quantidade de linhas: {iris.count()}")

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



Quantidade de linhas: 150


In [7]:
from pyspark.ml.feature import RFormula

Rformula = RFormula(formula="class ~ .", featuresCol="independente", labelCol="dependente")
irisrf = Rformula.fit(iris).transform(iris)
irisrf.select("independente", "dependente").show(5)

+-----------------+----------+
|     independente|dependente|
+-----------------+----------+
|[5.1,3.5,1.4,0.2]|       0.0|
|[4.9,3.0,1.4,0.2]|       0.0|
|[4.7,3.2,1.3,0.2]|       0.0|
|[4.6,3.1,1.5,0.2]|       0.0|
|[5.0,3.6,1.4,0.2]|       0.0|
+-----------------+----------+
only showing top 5 rows



In [10]:
irisTreino, irisTeste = irisrf.randomSplit([0.7, 0.3])

print(irisTreino.count())
print(irisTeste.count())

102
48


In [28]:
from pyspark.ml.classification import NaiveBayes

naive = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="independente", labelCol="dependente")
modelo = naive.fit(irisTreino)


In [29]:
previsao = modelo.transform(irisTeste)
previsao.select("dependente", "prediction").show()

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
+----------+----------+
only showing top 20 rows



In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction",
                                                metricName="accuracy")

acuracia = performance.evaluate(previsao)

print(f"Acurácia: {acuracia}")

Acurácia: 0.875
